In [ ]:
from tensorflow.keras.layers import Layer, Lambda, Conv2D, Dropout,Dense,Activation,Input,GlobalAveragePooling1D, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.layers import Reshape,Flatten,BatchNormalization,MaxPooling1D,AveragePooling2D,Reshape,Attention, ReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tqdm.auto import tqdm
import os
from Config import Config
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Mean, SparseCategoricalAccuracy
from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import random
import warnings
warnings.filterwarnings('always')

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything()

In [ ]:
DATA_PATH = 'EMODB'
CLASS_LABELS = Config.EMODB_LABELS
k = Config.EMODB_K

model_name = 'LIGHT_SERNET'
feature_name = 'mfcc'

EPOCHS = 500
BATCH = 64

In [ ]:
class SERNET(tf.keras.Model):

  def __init__(self, num_classes, L2=1e-6, DROPOUT=0.3):
    super().__init__()

    self.path1 = Sequential([
        Conv2D(32, (11, 1), padding="same", strides=(1, 1)),
        BatchNormalization(),
        ReLU(),
        AveragePooling2D(pool_size=2, padding='same')
    ])
    self.path2 = Sequential([
        Conv2D(32, (1, 9), padding="same", strides=(1, 1)),
        BatchNormalization(),
        ReLU(),
        AveragePooling2D(pool_size=2, padding='same')
    ])
    self.path3 = Sequential([
        Conv2D(32, (3, 3), padding="same", strides=(1, 1)),
        BatchNormalization(),
        ReLU(),
        AveragePooling2D(pool_size=2, padding='same')
    ])
    
    
    #### LFLB Blocks
    self.conv1 = Conv2D(32, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    self.bn1 = BatchNormalization()
    self.relu1 = ReLU()
    self.max_pool1 = AveragePooling2D(pool_size=(2, 2), padding='same')
    self.top1 = Conv2D(64, (1, 1), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    self.down1 = Conv2D(32, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    
    self.conv2 = Conv2D(64, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    self.bn2 = BatchNormalization()
    self.relu2 = ReLU()
    self.max_pool2 = AveragePooling2D(pool_size=(2, 2), padding='same')
    self.top2 = Conv2D(64, (1, 1), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    self.down2 = Conv2D(32, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    
    self.conv3 = Conv2D(128, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    self.bn3 = BatchNormalization()
    self.relu3 = ReLU()
    self.max_pool3 = AveragePooling2D(pool_size=(2, 2), padding='same')
    self.top3 = Conv2D(64, (1, 1), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    self.down3 = Conv2D(32, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    
    self.conv4 = Conv2D(256, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    self.bn4 = BatchNormalization()
    self.relu4 = ReLU()
    self.max_pool4 = AveragePooling2D(pool_size=(2, 2), padding='same')
    self.top4 = Conv2D(64, (1, 1), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    self.down4 = Conv2D(32, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    
    self.conv5 = Conv2D(512, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    self.bn5 = BatchNormalization()
    self.relu5 = ReLU()
    self.max_pool5 = AveragePooling2D(pool_size=(2, 2), padding='same')
    self.top5 = Conv2D(64, (1, 1), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))
    self.down5 = Conv2D(32, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))

    self.feature = Conv2D(32*5, (3, 3), strides=1, padding='same', use_bias=False, kernel_regularizer=regularizers.l2(L2))

    self.gap = GlobalAveragePooling2D()
    self.drop = Dropout(DROPOUT)
    self.classifier = Dense(num_classes, activation="softmax")


  def call(self, inputs):
    x = inputs

    path1 = self.path1(x)
    path2 = self.path2(x)
    path3 = self.path3(x)

    x = tf.math.maximum(path1, path2)
    x = tf.math.maximum(x, path3)
    

    p1 = self.conv1(x)
    p1 = self.bn1(p1)
    p1 = self.relu1(p1)
    # p1 = self.max_pool1(p1)
    
    p2 = self.conv2(p1)
    p2 = self.bn2(p2)
    p2 = self.relu2(p2)
    # p2 = self.max_pool2(p2)
    
    p3 = self.conv3(p2)
    p3 = self.bn3(p3)
    p3 = self.relu3(p3)
    # p3 = self.max_pool3(p3)
    
    p4 = self.conv4(p3)
    p4 = self.bn4(p4)
    p4 = self.relu4(p4)
    # p4 = self.max_pool4(p4)
    
    p5 = self.conv5(p4)
    p5 = self.bn5(p5)
    p5 = self.relu5(p5)
    # p5 = self.max_pool5(p5)

    top5 = self.top5(p5)
    
    top4 = tf.math.maximum(self.top4(p4), top5)
    top3 = tf.math.maximum(self.top3(p3), top4)
    top2 = tf.math.maximum(self.top2(p2), top3)
    top1 = tf.math.maximum(self.top1(p1), top2)
    
    down1 = self.down1(top1)
    down2 = self.down2(top2)
    down3 = self.down3(top3)
    down4 = self.down4(top4)
    down5 = self.down5(top5)

    x = Concatenate(axis=1)([down1, down2, down3, down4, down5])
    x1 = tf.math.maximum(down1, down2)
    x2 = tf.math.maximum(down3, down4)
    x = tf.math.maximum(x1, x2)
    x = tf.math.maximum(x, down5)
    
    x = self.feature(x)
    
    x = self.gap(x)
    x = self.drop(x)
    output = self.classifier(x)

    return output

In [ ]:
def train_step(model, loss_fn, optimizer, x, labels):
    with tf.GradientTape() as tape:
        # 미분 계산
        predictions = model(x, training=True)
        loss = loss_fn(labels, predictions)
        
    grad = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grad, model.trainable_variables))     # 신경망 파라미터 업데이트
    
    acc = sum(np.squeeze(labels) == np.argmax(predictions, axis=1))
    
    return loss, acc/len(labels)*100

def test_step(model, loss_fn, x, labels):
    predictions = model(x)
    loss = loss_fn(labels, predictions)
    acc = sum(np.squeeze(labels) == np.argmax(predictions, axis=1))
    
    return loss, acc/len(labels)*100, predictions

In [ ]:
# Read data
with open(f'dataset/{DATA_PATH}.npy', 'rb') as f:
    x = np.load(f)
    y = np.load(f)

# y = to_categorical(y,num_classes=len(Config.CLASS_LABELS))

In [ ]:
def save_model(model, dataset_name, model_name, feature_name, fold, now_time):
    save_path = os.path.join('Models', dataset_name)
    os.makedirs(save_path, exist_ok=True)
    
    naming = f'{model_name}_{feature_name}_{fold}-fold_{now_time}'
    
    h5_path = f'{naming}.h5'
    model.save_weights(os.path.join(save_path, h5_path))

In [ ]:
LEARNING_RATE_DECAY_PARAMETERS = -0.15
LEARNING_RATE_DECAY_STRATPOINT = 50
LEARNING_RATE_DECAY_STEP = 20


def scheduler(epoch, lr):
    if epoch < LEARNING_RATE_DECAY_STRATPOINT:
        return lr
    else:
        if epoch % LEARNING_RATE_DECAY_STEP == 0:
            lr = lr * tf.math.exp(LEARNING_RATE_DECAY_PARAMETERS)
    return lr

In [ ]:
conf_matrix_list = []
eva_matrix = []

avg_acc = 0.0

In [ ]:
import matplotlib.pyplot as plt

kfold = KFold(n_splits=k, shuffle=False, random_state=None)
for i, (train, test) in tqdm(enumerate(kfold.split(x, y)), desc='Training {k}-Fold.....'):
    now_time = datetime.now().strftime("%m-%d-%H%M%S")
    
    x_train, y_train = x[train], y[train]
    # y[train] = smooth_labels(y[train], 0.1)
    
    x_test, y_test = x[test], y[test]
    
    shape = x_train.shape[1:]
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = Adam(learning_rate=1e-5)
    
    model = SERNET(len(CLASS_LABELS))
    
    best_test_loss = 0x3f3f3f
    best_test_acc = -1
    
    epoch_losses = []
    valid_losses = []
    
    batch_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(2022).batch(BATCH)
    batch_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH)
    for epoch in tqdm(range(EPOCHS), desc=f'Fold-{i+1}'):

        train_loss, train_acc = [], []
        for features, labels in batch_train:
            loss, acc = train_step(model, loss_fn, optimizer, features, labels)
            train_loss.append(loss)
            train_acc.append(acc)
            
        test_loss, test_acc = [], []
        for features, labels in batch_test:
            loss, acc, _ = test_step(model, loss_fn, features, labels)
            test_loss.append(loss)
            test_acc.append(acc)
        
        epoch_loss = sum(train_loss)/len(train_loss)
        epoch_acc = sum(train_acc)/len(train_acc)
        val_loss = sum(test_loss)/len(test_loss)
        val_acc = sum(test_acc)/len(test_acc)
        
        epoch_losses.append(epoch_loss)
        valid_losses.append(val_loss)
        
        cur_lr = K.eval(optimizer.lr)
        print(f'{epoch+1}/{EPOCHS} lr={cur_lr:.5f} - loss:{epoch_loss:.3f}, acc:{epoch_acc:.3f}, val_loss:{val_loss:.3f}, val_acc:{val_acc:.3f}')
        print(f'Best loss:{best_test_loss:.3f}, Best accuracy:{best_test_acc:.3f}')
        
        set_lr = scheduler(epoch, K.eval(optimizer.lr))
        K.set_value(optimizer.learning_rate, set_lr)
        
        if best_test_loss > val_loss:
            best_test_acc = val_acc
            best_test_loss = val_loss
            y_pred_best = model.predict(x[test])
            
            # model save
            save_model(model, DATA_PATH, model_name, feature_name, i, now_time)
            
    print(f'[*] Done - acc:{best_test_acc:.3f}')
    
    plt.title('Loss Curve')
    plt.ylabel('Loss', fontsize=16)
    plt.xlabel('Epoch', fontsize=16)
    plt.plot(epoch_losses[:],'b')
    plt.plot(valid_losses[:],'r')
    plt.legend(['Training loss','Validation loss'])
    plt.show()
    
    avg_acc += best_test_acc
    
    

In [ ]:
avg_acc/k

In [ ]:
print(emotions_groundtruth_list)
print(predicted_emotions_list)

In [ ]:
Report = classification_report(emotions_groundtruth_list, predicted_emotions_list)

os.makedirs(f'Results/{DATA_PATH}', exist_ok=True)
report_path = f'Results/{DATA_PATH}/{model_name}_{feature_name}_{k}-fold_nomalize.txt'

with open(report_path, "w") as f:
    f.write(Report)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

emotion_names = Config.CLASS_LABELS


# build confusion matrix and normalized confusion matrix
conf_matrix = confusion_matrix(emotions_groundtruth_list, predicted_emotions_list)
conf_matrix_norm = confusion_matrix(emotions_groundtruth_list, predicted_emotions_list,normalize='true')

# make a confusion matrix with labels using a DataFrame
confmatrix_df = pd.DataFrame(conf_matrix, index=emotion_names, columns=emotion_names)
confmatrix_df_norm = pd.DataFrame(conf_matrix_norm, index=emotion_names, columns=emotion_names)

# plot confusion matrices
plt.figure(figsize=(16,6))
sn.set(font_scale=1.8) # emotion label and title size
plt.subplot(1,2,1)
plt.title('Confusion Matrix')
sn.heatmap(confmatrix_df, annot=True, annot_kws={"size": 13}, fmt='g') #annot_kws is value font
plt.subplot(1,2,2)
plt.title('Normalized Confusion Matrix')
sn.heatmap(confmatrix_df_norm, annot=True, annot_kws={"size": 13}) #annot_kws is value font
plt.savefig(f'Results/{DATA_PATH}/{model_name}_{feature_name}_{k}-fold_confmatrix.pdf')
plt.show()

In [ ]:
import pandas as pd

naming = f'Results/{DATA_PATH}/{model_name}_{feature_name}_{k}-fold'
naming = f'{naming}_{avg_acc/5:.3f}.xlsx'

writer = pd.ExcelWriter(naming)
for i,item in enumerate(conf_matrix_list):
    temp = {}
    temp[" "] = CLASS_LABELS
    j = 0
    for j,l in enumerate(item):
        temp[CLASS_LABELS[j]]=item[j]
    data1 = pd.DataFrame(temp)
    data1.to_excel(writer,sheet_name=str(i), encoding='utf8')
    df = pd.DataFrame(eva_matrix[i]).transpose()
    df.to_excel(writer,sheet_name=str(i)+"_evaluate", encoding='utf8')
writer.save()

